In [1]:
import numpy as np
from tqdm import tqdm

data = np.load('./data/eastmoney_full_stocks_list_nlu_tencent.pkl', allow_pickle=True)

In [3]:
data[0]['text_keyword']

[{"Score": 1.0, "Word": "公司"},
 {"Score": 0.88982016, "Word": "医疗器械"},
 {"Score": 0.69142354, "Word": "浙江省"},
 {"Score": 0.6476816, "Word": "针"},
 {"Score": 0.619902, "Word": "注射针"},
 {"Score": 0.6144252, "Word": "制造"},
 {"Score": 0.58027697, "Word": "技术"},
 {"Score": 0.57011515, "Word": "产品"},
 {"Score": 0.56897545, "Word": "用户"},
 {"Score": 0.56668705, "Word": "品种"},
 {"Score": 0.55956155, "Word": "行业"},
 {"Score": 0.5518364, "Word": "康德莱集团"},
 {"Score": 0.5193484, "Word": "单位"},
 {"Score": 0.502113, "Word": "企业"},
 {"Score": 0.45472413, "Word": "高分子"},
 {"Score": 0.45368028, "Word": "分会"},
 {"Score": 0.45321786, "Word": "理事长"},
 {"Score": 0.39482775, "Word": "高新技术"},
 {"Score": 0.3924612, "Word": "厂家"},
 {"Score": 0.36895323, "Word": "试点"},
 {"Score": 0.36446825, "Word": "称号"},
 {"Score": 0.35745487, "Word": "发展"},
 {"Score": 0.34765783, "Word": "融合"},
 {"Score": 0.33645958, "Word": "示范"},
 {"Score": 0.3358695, "Word": "制造业"},
 {"Score": 0.33228558, "Word": "系列"},
 {"Score": 0.32576

In [4]:
keywords_all = []

for one in data:
    for item in one['text_keyword']:
        keywords_all.append(item.Word)

In [5]:
len(keywords_all)

14674

In [6]:
keywords_set = list(set(keywords_all))

In [7]:
len(keywords_set)

4757

In [12]:
all = []

for one in tqdm(data):
    vector = np.zeros(len(keywords_set))
    for item in one['text_keyword']:
        word = item.Word
        idx = keywords_set.index(word)
        vector[idx] = 1
    all.append(
        {
            'ticker_id': one['ticker_id'],
            'ticker_name': one['ticker_name'],
            'vector': vector,
        }
    )


import pickle

with open('./data_ignore/vectors_keyword_Bag.pkl','wb') as f:
    pickle.dump(all, f)

100%|██████████| 510/510 [00:00<00:00, 819.47it/s]


In [13]:
import numpy as np
from tqdm import tqdm

data = np.load('./data_ignore/vectors_keyword_Bag.pkl', allow_pickle=True)

In [14]:
data[0]['vector']

array([0., 0., 0., ..., 0., 0., 0.])

In [15]:
ticker_names = []
ticker_ids = []
vectors = []

for idx in tqdm(range(len(data))):
    if 'vector' in data[idx].keys():
        vectors.append(data[idx]['vector'])
        ticker_ids.append(data[idx]['ticker_id'])
        ticker_names.append(data[idx]['ticker_name'])

vectors = np.stack(vectors)

100%|██████████| 510/510 [00:00<00:00, 326729.04it/s]


In [16]:
def similarity_vector_matrix(arr, brr):
    return arr.dot(brr.T) / (np.sqrt(np.sum(arr*arr)) * np.sqrt(np.sum(brr*brr, axis=1)))

In [17]:
import jieba


def stock_search(query, topk=10):

    tokens = jieba.lcut(query)
    vector = np.zeros(len(keywords_set))
    for token in tokens:
        try:
            idx = keywords_set.index(token)
            vector[idx] = 1
        except Exception:
            pass

    res = similarity_vector_matrix(vector, vectors)
    idxs = np.argsort(res)[::-1]

    topk_idxs = idxs[:topk]
    names = [ticker_names[idx] for idx in topk_idxs]

    print(names)

- 自动驾驶，新能源汽车
- 电影，电视剧，文化艺术
- 啤酒，烧烤，朋友聚会
- 医疗保险，重大疾病保障
- 新冠肺炎

In [19]:
query = '自动驾驶，新能源汽车'
stock_search(query)

['阿尔特', '杉杉股份', '宁德时代', '上汽集团', '双环传动', '比亚迪', '三峡能源', '天齐锂业', '华阳集团', '天奇股份']


In [20]:
query = '电影，电视剧，文化艺术'
stock_search(query)

['光线传媒', '中国建筑', '水井坊', '孩子王', '中科电气', '新城控股', '东方雨虹', '泸州老窖', '太阳纸业', '索菲亚']


In [21]:
query = '啤酒，烧烤，朋友聚会'
stock_search(query)

['重庆啤酒', '燕京啤酒', '来伊份', '珠江啤酒', '青岛啤酒', '华阳国际', '中科电气', '药明康德', '爱尔眼科', '长城汽车']


In [22]:
query = '医疗保险，重大疾病保障'
stock_search(query)

['广和通', '科威尔', '中国船舶', '爱尔眼科', '华阳集团', '小熊电器', '中科电气', '药明康德', '长城汽车', '爱施德']


In [23]:
query = '新冠肺炎'
stock_search(query)

['康希诺', '中国船舶', '千味央厨', '天赐材料', '华策影视', '赞宇科技', '伟星股份', '美的集团', '华阳国际', '长城汽车']
